In [1]:
import pandas as pd
import numpy as np
import torch

from eugene.dataload._io import read_numpy, read

# Autoreload extension
if 'autoreload' not in get_ipython().extension_manager.loaded:
    %load_ext autoreload
%autoreload 2

# Basic import
import eugene as eu
eu.__version__

Global seed set to 13


GPU is available: True
Number of GPUs: 1
Current GPU: 0
GPUs: NVIDIA GeForce RTX 2070


'0.0.6'

In [2]:
sdata = read_numpy("chr1.npy_ohe_seqs.npy", ohe=True)

In [3]:
eu.pp.train_test_split_sdata(sdata)

SeqData object modified:
    seqs_annot:
        + train_val


In [4]:
seq_len = 50
latent_dim = seq_len * 4
generator = torch.nn.Sequential(
    eu.models.base.BasicFullyConnectedModule(
        input_dim=500,
        output_dim=latent_dim,
        hidden_dims=[300]
    ),
    torch.nn.Tanh()
)
discriminator = torch.nn.Sequential(
    eu.models.base.BasicFullyConnectedModule(
        input_dim=latent_dim,
        output_dim=1,
        hidden_dims=[100]
    ),
    torch.nn.Sigmoid()
)

In [5]:
model = eu.models.GAN(
    latent_dim=latent_dim, 
    generator=generator, 
    discriminator=discriminator,
    gen_lr=1e-3,
    disc_lr=1e-3,
)

In [8]:
eu.settings.batch_size = 512
eu.settings.dl_num_workers = 8
eu.settings.dl_pin_memory_gpu_training = True
eu.train.fit(
    model=model,
    sdata=sdata,
)

Global seed set to 13
GPU available: True, used: True


No transforms given, assuming just need to tensorize.
No transforms given, assuming just need to tensorize.


TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name          | Type       | Params
---------------------------------------------
0 | generator     | Sequential | 210 K 
1 | discriminator | Sequential | 20.2 K
---------------------------------------------
230 K     Trainable params
0         Non-trainable params
230 K     Total params
0.923     Total estimated model params size (MB)


Validation sanity check: 0it [00:00, ?it/s]

Global seed set to 13


torch.Size([512, 4, 50]) cuda:0
torch.Size([512, 4, 50]) cuda:0


Training: 0it [00:00, ?it/s]

OSError: [Errno 22] Invalid argument

---